<a href="https://colab.research.google.com/github/henripl12/trading/blob/main/Neurual_networks_models_mit_funktionen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import yfinance as yf
import joblib
from sklearn.neural_network import MLPClassifier
from dateutil.relativedelta import relativedelta


def train_model(symbol):
    # Download historical prices of SPY
    symbol = 'SPY'
    start_date = start_date = dt.datetime.now() -relativedelta(years= 23)
    end_date = dt.datetime.now()
    all_data = yf.download(symbol, start=start_date, end=end_date, interval="1d")
    # add start_date as rolling e.g. last 20 years to train

    # Calculate technical indicators
    all_data['SMA_5'] = all_data['Close'].rolling(window=5).mean()
    all_data['SMA_15'] = all_data['Close'].rolling(window=15).mean()
    all_data['SMA_ratio'] = all_data['SMA_15'] / all_data['SMA_5']
    all_data['15MA'] = all_data['Close'].rolling(window=15).mean()
    all_data['SD'] = all_data['Close'].rolling(window=15).std()
    all_data['upperband'] = all_data['15MA'] + 2 * all_data['SD']
    all_data['lowerband'] = all_data['15MA'] - 2 * all_data['SD']
    all_data['return'] = all_data['Close'].pct_change()

    # Prepare data for backtesting (same as your original code)
    all_data['Target_Direction'] = np.where(all_data['return'] > 0, 1, 0)
    all_data = all_data.dropna().copy()
    Target_variables = ['SMA_ratio', 'SMA_5', 'SMA_15', '15MA', 'SD', 'upperband', 'lowerband']

    # Split data into train and test sets (same as your original code)
    X = all_data[Target_variables]
    y = all_data['Target_Direction']
    train_size = int(0.75 * len(all_data))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the model (same as your original code)
    clf =  MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50, 10,50), random_state=1)
    model= clf.fit(X_train, y_train)

    # Save the trained model for future use
    joblib.dump(model, 'trained_model.pkl')



In [3]:
# Load the trained model and data to predict
def load_model_and_predict(symbol):
    clf = joblib.load('trained_model.pkl')

    # Download historical prices of SPY
    symbol
    start_date = start_date = dt.datetime.now() -relativedelta(days= 30)
    end_date = dt.datetime.now()
    all_data = yf.download(symbol, start=start_date, end=end_date, interval="1d")
    # add start_date as rolling e.g. last 20 years to train

    # Calculate technical indicators
    all_data['SMA_5'] = all_data['Close'].rolling(window=5).mean()
    all_data['SMA_15'] = all_data['Close'].rolling(window=15).mean()
    all_data['SMA_ratio'] = all_data['SMA_15'] / all_data['SMA_5']
    all_data['15MA'] = all_data['Close'].rolling(window=15).mean()
    all_data['SD'] = all_data['Close'].rolling(window=15).std()
    all_data['upperband'] = all_data['15MA'] + 2 * all_data['SD']
    all_data['lowerband'] = all_data['15MA'] - 2 * all_data['SD']
    all_data['return'] = all_data['Close'].pct_change()

    # Prepare data for backtesting (same as your original code)
    all_data = all_data.dropna().copy()
    Target_variables = ['SMA_ratio', 'SMA_5', 'SMA_15', '15MA', 'SD', 'upperband', 'lowerband']

    # Split data into train and test sets (same as your original code)
    test_data = all_data.iloc[-1:]
    X = all_data[Target_variables]
    X_test =  X[-1:]

    y_pred =  clf.predict(X_test)
    test_data['Predicted_Direction'] = y_pred
    test_data['Signal'] = np.where(test_data['Predicted_Direction'] == 1, 'Buy', 'Sell')

    date_of_prediction = all_data.index[-1]

    # Create a DataFrame with predictions and other relevant data
    predictions_data = pd.DataFrame({
      'Date': [date_of_prediction],
      'Predicted_Direction': y_pred,
      'Signal': ['Buy' if pred == 1 else 'Sell' for pred in y_pred]
    })
    # Save predictions to a CSV file
    file_path = 'predictions.csv'
    predictions_data.to_csv(file_path, mode='a', header=False, index=False)

In [4]:
train_model('SPY')
load_model_and_predict('SPY')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-3-1672966ecd94>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Predicted_Direction'] = y_pred
<ipython-input-3-1672966ecd94>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Signal'] = np.where(test_data['Predicted_Direction'] == 1, 'Buy', 'Sell')
